# Recommendation Engine for Movie_lens data (lab41 dataset)

In [3]:
import numpy as np
import pandas as pd
from scipy import stats as st
import matplotlib.pyplot as plt

In [4]:
ratings = pd.read_table("C:\\Users\\Lenovo\\Desktop\\mlstartup\\ml-1m\\ratings.dat", sep= '::', names=['userid', 'movieid', 'rating', 'timestamp'])

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


In [5]:
movies = pd.read_table("C:\\Users\\Lenovo\\Desktop\\mlstartup\\ml-1m\\movies.dat", sep= '::', names=['movieid', 'title', 'genre'])

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


In [13]:
movies.head()

,movieid,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
users = pd.read_table("C:\\Users\\Lenovo\\Desktop\\mlstartup\\ml-1m\\users.dat", sep= '::', names=['userid', 'gender', 'age', 'occupation','zipcode'])

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


### data information

In [58]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 3 columns):
movieid    3883 non-null int64
title      3883 non-null object
genre      3883 non-null object
dtypes: int64(1), object(2)
memory usage: 91.1+ KB


In [59]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040 entries, 0 to 6039
Data columns (total 5 columns):
userid        6040 non-null int64
gender        6040 non-null object
age           6040 non-null int64
occupation    6040 non-null int64
zipcode       6040 non-null object
dtypes: int64(3), object(2)
memory usage: 236.0+ KB


In [60]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 4 columns):
userid       1000209 non-null int64
movieid      1000209 non-null int64
rating       1000209 non-null int64
timestamp    1000209 non-null int64
dtypes: int64(4)
memory usage: 30.5 MB


### missing values analysis

In [25]:
movies.isnull().sum()

movieid    0
title      0
genre      0
dtype: int64

In [27]:
ratings.isnull().sum()

user_id      0
item_id      0
rating       0
timestamp    0
dtype: int64

In [29]:
users.isnull().sum()

user_id       0
gender        0
age           0
occupation    0
zipcode       0
dtype: int64

### Analyzing user profile and information will be used to increase model performance 

In [50]:
group_age = users.groupby('age')
group_age.agg('count')

,user_id,gender,occupation,zipcode
age,,,,
1,222,222,222,222
18,1103,1103,1103,1103
25,2096,2096,2096,2096
35,1193,1193,1193,1193
45,550,550,550,550
50,496,496,496,496
56,380,380,380,380


In [51]:
group_user_gender = users.groupby('gender')
group_user_gender.agg('count')

,user_id,age,occupation,zipcode
gender,,,,
F,1709,1709,1709,1709
M,4331,4331,4331,4331


In [52]:
group_user_occupation = users.groupby('occupation')
group_user_occupation.agg('count')

,user_id,gender,age,zipcode
occupation,,,,
0,711,711,711,711
1,528,528,528,528
2,267,267,267,267
3,173,173,173,173
4,759,759,759,759
5,112,112,112,112
6,236,236,236,236
7,679,679,679,679
8,17,17,17,17


In [54]:
group_user_age_gender = users.groupby(['age','gender'])
group_user_age_gender.agg('count')

user_id  occupation  zipcode
age gender                              
1   F            78          78       78
    M           144         144      144
18  F           298         298      298
    M           805         805      805
25  F           558         558      558
    M          1538        1538     1538
35  F           338         338      338
    M           855         855      855
45  F           189         189      189
    M           361         361      361
50  F           146         146      146
    M           350         350      350
56  F           102         102      102
    M           278         278      278

In [61]:
#merging the users data with the ratings
user_ratings=ratings.merge(users, how='left', on='userid' )

In [72]:
user_ratings_userid = user_ratings.groupby('userid')
avg_user_rating= user_ratings_userid.agg('mean')

In [66]:
user_ratings_age = user_ratings.groupby('age')
user_ratings_age.agg(['mean', 'count', 'std'])

userid                           movieid                       \
            mean   count          std         mean   count          std   
age                                                                       
1    2717.664437   27211  1755.976936  1869.380875   27211  1107.206394   
18   2789.483818  183536  1735.479473  1815.417967  183536  1099.670860   
25   3121.309319  395556  1734.429301  1849.590119  395556  1094.961643   
35   3040.853475  199003  1689.068974  1895.714251  199003  1089.686193   
45   3178.966138   83633  1784.595472  1903.871127   83633  1092.457322   
50   3016.087543   72490  1668.200827  1917.925962   72490  1096.612511   
56   2963.619082   38780  1661.022577  1927.316194   38780  1106.875332   

       rating                    timestamp                       occupation  \
         mean   count       std       mean   count           std       mean   
age                                                                           
1    3.549520   27211  1.208417  975979327   27211  1.617570e+07   8.765977   
18   3.507573  183536  1.165970  973691826  183536  1.298613e+07   6.725596   
25   3.545235  395556  1.127175  972056761  395556  1.243697e+07   7.887353   
35   3.618162  199003  1.078101  971559281  199003  1.083324e+07   8.816138   
45   3.638062   83633  1.065385  970821046   83633  1.113813e+07   8.531321   
50   3.714512   72490  1.061380  972227771   72490  1.181067e+07   8.359098   
56   3.766632   38780  1.062551  971285559   38780  9.558104e+06   9.569856   

                       
      count       std  
age                    
1     27211  4.515105  
18   183536  5.971305  
25   395556  6.698193  
35   199003  6.606228  
45    83633  6.682915  
50    72490  6.718466  
56    38780  6.323344

In [67]:
user_ratings_gender = user_ratings.groupby('gender')
user_ratings_gender.agg(['mean', 'count', 'std'])

userid                           movieid                       \
              mean   count          std         mean   count          std   
gender                                                                      
F       3130.43738  246440  1791.592029  1824.086212  246440  1096.602480   
M       2989.88083  753769  1705.824434  1879.092918  753769  1095.517229   

          rating                    timestamp                        \
            mean   count       std       mean   count           std   
gender                                                                
F       3.620366  246440  1.111228  972432737  246440  1.347125e+07   
M       3.568879  753769  1.118724  972181889  753769  1.168854e+07   

              age                    occupation                    
             mean   count        std       mean   count       std  
gender                                                             
F       29.803855  246440  12.144116   6.722837  246440  6.229740  
M       29.716885  753769  11.620835   8.465514  753769  6.570269

In [70]:
user_ratings_imp = user_ratings[['userid','age','movieid','rating','gender']].copy()


,userid,age,movieid,rating,gender
0,1,1,1193,5,F
1,1,1,661,3,F
2,1,1,914,3,F
3,1,1,3408,4,F
4,1,1,2355,5,F


In [74]:
avg_user_rating.head()

,movieid,rating,timestamp,age,occupation
userid,,,,,
1,1560.547170,4.188679,9.784297e+08,1.0,10.0
2,1784.015504,3.713178,9.782993e+08,56.0,16.0
3,1787.450980,3.901961,9.782978e+08,25.0,15.0
4,1932.000000,4.190476,9.782942e+08,45.0,7.0
5,1762.747475,3.146465,9.782445e+08,25.0,20.0


### creating user id and movie matrix

In [79]:
ratings.head()

,userid,movieid,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
unique_users = ratings.userid.unique().shape[0]
unique_movies = ratings.movieid.unique().shape[0]

In [7]:
ratings_prep = ratings[['userid','movieid','rating']].copy()

In [8]:
from sklearn.cross_validation import train_test_split
train, test = train_test_split(ratings_prep, test_size = 0.2, random_state = 42)


In [10]:
train_user_movie = train.pivot(index='userid', columns='movieid', values='rating')

In [11]:
test_user_movie =test.pivot(index='userid', columns='movieid', values='rating')

In [148]:
train_user_movie.head()

movieid,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
userid,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
train_user_movie_impute = train_user_movie.fillna(0)

In [158]:
train_user_movie_impute.head()

movieid,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
userid,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
numpyMatrix = train_user_movie_impute.as_matrix()

### Calculating cosine simlarity between users 

In [133]:
def fast_similarity(ratings, kind='user', epsilon=1e-9):
    # epsilon -> small number for handling dived-by-zero errors
    if kind == 'user':
        sim = ratings.dot(ratings.T) + epsilon
    elif kind == 'item':
        sim = ratings.T.dot(ratings) + epsilon
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return (sim / norms / norms.T)

In [15]:
sim =numpyMatrix.dot(numpyMatrix.T) + 1e-9
norms = np.array([np.sqrt(np.diagonal(sim))])
test_sim= sim/norms/norms.T

In [16]:
similarity_matrix=pd.DataFrame(test_sim)

In [17]:
similarity_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,6030,6031,6032,6033,6034,6035,6036,6037,6038,6039
0,1.000000,0.051661,0.133699,0.080533,0.036697,1.857391e-01,0.072645,0.052055,0.240745,0.229071,...,0.115598,4.889341e-02,0.052172,4.449986e-02,0.087569,0.182107,0.099235,2.300438e-12,0.177446,0.116286
1,0.051661,1.000000,0.132976,0.132501,0.104168,1.085479e-01,0.297586,0.174417,0.164335,0.192822,...,0.129935,2.902738e-02,0.267718,1.783281e-02,0.161510,0.197579,0.164144,5.377600e-02,0.059562,0.195561
2,0.133699,0.132976,1.000000,0.084733,0.031854,5.657088e-02,0.110404,0.057509,0.120513,0.159982,...,0.059331,6.859136e-02,0.168697,2.341041e-12,0.051563,0.122201,0.077493,1.089189e-01,0.069893,0.051450
3,0.080533,0.132501,0.084733,1.000000,0.039140,1.737751e-12,0.144631,0.055520,0.061836,0.078437,...,0.112268,1.112495e-01,0.325729,3.164145e-12,0.067407,0.150780,0.088201,3.271435e-02,0.020705,0.126359
4,0.036697,0.104168,0.031854,0.039140,1.000000,3.895974e-02,0.102939,0.214950,0.190200,0.077233,...,0.079593,7.216935e-13,0.075225,1.773474e-02,0.146214,0.238963,0.139038,1.986410e-02,0.009671,0.195674


### Predicting (estimating) the ratings by a user for each movie

###### The strategy: 
Identifying the top 10 most similar users to every users and then calculate the weighted average of the users ratings to estimate a probable rating by a user for each movie. Then, as required, recommend the top 10/top 20 movies.


In [ ]:
#creating an empty matrix to populate with predictions 

In [30]:
recommendation_matrix = np.zeros(numpyMatrix.shape)

In [ ]:
# populating the recommendation matrix based on 10 similar users

In [ ]:
for i in range(numpyMatrix.shape[0]):
    similar_user_index = [np.argsort(test_sim[:,i])[:-11-1:-1]]
    for j in range(numpyMatrix.shape[1]):
        recommendation_matrix[i, j] = test_sim[i, :][similar_user_index].dot(numpyMatrix[:, j][similar_user_index]) 
        recommendation_matrix[i, j] /= np.sum(np.abs(test_sim[i, :][similar_user_index]))


In [ ]:
# using the recommendation matrix to provide recommendation

In [115]:
def recommendation_engine(userid=1, number_of_recommendation=10):
    if type(userid)!=int or number_of_recommendation<1:
        print('Please enter integer user id and number of recommendation')
        import sys
        sys.exit
    if userid>6040 or userid<1 or number_of_recommendation>3680:
        print('please enter userid between 1 and 6040 and recommendations')
        import sys
        sys.exit
    recommend_list=[]
    userid-=1
    n=number_of_recommendation+1
    recommendations =np.argsort(recommendation_matrix[userid,:])[:-n:-1]
    #print(len(recommendations.tolist()))
    for i in recommendations.tolist():
        i+=1
        #print(i)
        index=np.where(movies.movieid==i)[0]
        title=movies.iloc[index,1]
        #print(title)
        recommend_list.append(title)
    return recommend_list    

In [117]:
a=recommendation_engine(1,10)
a

[567    Wedding Gift, The (1994)
 Name: title, dtype: object, 574    Hour of the Pig, The (1993)
 Name: title, dtype: object, 842    Dingo (1992)
 Name: title, dtype: object, 1669    Stranger in the House (1997)
 Name: title, dtype: object, 1157    Cook the Thief His Wife & Her Lover, The (1989)
 Name: title, dtype: object, 2082    Gods Must Be Crazy II, The (1989)
 Name: title, dtype: object, 0    Toy Story (1995)
 Name: title, dtype: object, 573    Andre (1994)
 Name: title, dtype: object, 940    Around the World in 80 Days (1956)
 Name: title, dtype: object, 1007    Cool Runnings (1993)
 Name: title, dtype: object]

###Conclusion


 This approach will be prone to the following problems:
- No recommendations for new customers
- Users standards for the movies will result in easy vs critical raters. Some will always rate high and some will rate low.
- Users who always rate 5 for all movies and have rated many movies will always be similar to a user and will add bias to estimations.
- Movies which are present in the movies database but have never been rated ie, no present in the ratings database will not be rated

###Potential Solutions

- Clustering Based on demographics and identifying and recommending popular choices in a cluster to a new customer whose properties are similar to cluster characteristics
- Advanced solutions using matrix factorization algorithms can be used to include more features such as genre of the movie and other user related properties
- Seperate algorithms for new and existing customers can be used